<a href="https://colab.research.google.com/github/Jenni-Hawk/Advanced_Topic_Modeling/blob/main/mynext_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERTopic Modeling**

In [130]:
%%capture
!pip install bertopic

# Data

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
from google.colab import files
uploaded = files.upload()

Saving TweetBatch3d.csv to TweetBatch3d.csv


In [133]:
import pandas as pd
import io

tweets = pd.read_csv(io.BytesIO(uploaded['TweetBatch3d.csv']))

In [134]:
tweets.head()

,text,cleaned
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember and forthepeople and votebluein2022...
1,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
2,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
3,RT @tleehumphrey: Today is the beginning of th...,rt today is the beginning of the inquiry into ...
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,rt mitch mcconnell kevin mccarthy they both kn...


In [135]:
tweets.drop(['cleaned'], axis=1, inplace=True)

In [136]:
tweets.head()

,text
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...
1,RT @sandibachom: IS THIS THING ON???!!This is ...
2,RT @sandibachom: IS THIS THING ON???!!This is ...
3,RT @tleehumphrey: Today is the beginning of th...
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...


In [137]:
tweets.shape

(34993, 1)

In [138]:
#turn tweet column into a list of strings
tweet_list = tweets["text"].tolist()

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.

In this code we're also:
- Transforming documents to embeddings
  - default embedding model in BERTopic ("all-MiniLM-L6-v2")
  - If you want to use a model that provides a higher quality, but takes more computing time, then I would advise using all-mpnet-base-v2 works great for English documents. From his FAQs
- Reducing dimensionality
- Clustered reduced embeddings


In [139]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

# Fit Model to Data
# topics = each piece of content is assigned a cluster number
# probs = each piece of content assigned a probability number
topics, probs = topic_model.fit_transform(tweet_list)

Batches:   0%|          | 0/1094 [00:00<?, ?it/s]

2023-08-14 22:02:39,630 - BERTopic - Transformed documents to Embeddings
2023-08-14 22:04:24,061 - BERTopic - Reduced dimensionality
2023-08-14 22:16:27,092 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

In [140]:
#topics

In [141]:
#probs

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [142]:
freq = topic_model.get_topic_info(); freq.head(8)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2473,-1_january6thcommitteehearings_she_are_and,"[january6thcommitteehearings, she, are, and, f...",[RT @fugitivemama: She knew of the FBI operati...
1,0,6016,0_both_they_backed_kevin,"[both, they, backed, kevin, mitch, mccarthy, m...",[RT @AdamKinzinger: Mitch McConnell.\nKevin Mc...
2,1,1612,1_creating_hamill_correct_overthrowing,"[creating, hamill, correct, overthrowing, resi...",[RT @Resist_MAGA_GOP: Mark Hamill is correct. ...
3,2,1176,2_demands_ja_deserves_unanimously,"[demands, ja, deserves, unanimously, history, ...",[RT @AdamKinzinger: We just voted unanimously ...
4,3,1102,3_onthis_deploy_defense_pathetic,"[onthis, deploy, defense, pathetic, sec, actin...",[RT @sandibachom: IS THIS THING ON???!!This is...
5,4,1052,4_goaded_author_excuse_summoned,"[goaded, author, excuse, summoned, rioters, at...",[RT @AdamKinzinger: Trump is the author of the...
6,5,954,5_rejection_loss_break_couldnt,"[rejection, loss, break, couldnt, accept, trie...",[RT @AdamKinzinger: When he couldn't accept hi...
7,6,895,6_rudy_hatch_giuliani_meadows,"[rudy, hatch, giuliani, meadows, yet, new, rog...",[RT @StandForBetter: 📺 NEW Video:\n\nTrump Los...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [143]:
freq.shape

(375, 5)

get top 3 most frequent topics

In [144]:
topic_model.get_topic(0)  # Select the most frequent topic

[('both', 0.014137172499952283),
 ('they', 0.013163573083235795),
 ('backed', 0.012943021189628794),
 ('kevin', 0.012923096563650276),
 ('mitch', 0.012914785285151949),
 ('mccarthy', 0.012913725083226245),
 ('mcconnell', 0.012862417296169928),
 ('responsible', 0.012817727614188777),
 ('called', 0.012790010433921441),
 ('down', 0.01248235084334251)]

In [145]:
topic_model.get_topic(1)

[('creating', 0.0429185497674007),
 ('hamill', 0.0429185497674007),
 ('correct', 0.042900006172023324),
 ('overthrowing', 0.04282599961367145),
 ('resistmagagop', 0.042351407346683426),
 ('january6thcomm', 0.04141728687095705),
 ('love', 0.040890614897517635),
 ('without', 0.039862223753059554),
 ('country', 0.037254866582341624),
 ('violence', 0.026951878727700808)]

In [146]:
topic_model.get_topic(2)

[('demands', 0.045447992128299336),
 ('ja', 0.0452605947374387),
 ('deserves', 0.04377414251397236),
 ('unanimously', 0.043319831513733066),
 ('history', 0.041805932545065316),
 ('testify', 0.04014616338668143),
 ('subpoena', 0.03479444532519411),
 ('voted', 0.034265940301573365),
 ('oath', 0.031703343862431704),
 ('just', 0.02854256465560467)]

**NOTE**: BERTopic is stocastich which means that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **KeyBERT-Inspired Model**
- Reduce the appearance of stop words. This also often improves the topic representation:
- https://maartengr.github.io/BERTopic/api/representation/keybert.html
- https://maartengr.github.io/BERTopic/getting_started/representation/representation.html

In [147]:
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic

# Create your representation model
representation_model = KeyBERTInspired()

# Use the representation model in BERTopic on top of the default pipeline
topic_model2 = BERTopic(representation_model=representation_model)

In [148]:
topic_model2

#### Next Steps
- Fit this model, look over and generate visualizations, collect thoughts. Does this look better than the other?

# **Visualization**
Conduct these visualizations:
Visualizations: generate these for keybert and openai
- document visualization
https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_documents.html

- Term one (not bad) do this one too
https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_terms.html

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [149]:
topic_model.visualize_barchart(top_n_topics=5)

In [150]:
topic_model.visualize_topics()